<a href="https://colab.research.google.com/github/VinoJose/Tensorflow/blob/main/07_food_vision_milestone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Check GPU

Google colab offers free GPUs, however not all of them are compatible with mixed precisoion training.

Google colab offers:
* K80
* P100
* Tesla T4

Only Tesla T4 is compatible with mixed precision training 

In [2]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-00d6a704-695b-85e6-1669-3de5af71e26a)


In [3]:
# Download helper function script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-01-20 13:21:20--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-01-20 13:21:21 (71.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
# Import helper functions 
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

 ## Use TensorFlow datasets to download the data

In [5]:
# Import library for tensoflow datasets
import tensorflow_datasets as tfds

In [6]:
# List all available datasets
dataset_list = tfds.list_builders()
print("food101" in dataset_list)

True


In [8]:
# Load in the data
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train", "validation"],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

## Exploring the Food101 data from tfds

To become one with our data, we want to find:
* Class names
* The shape of the input data(image tensors)
* The datatype
* What the labels looks like (One hot encoded or label encoded)
* Do the labels match up with the class names?

In [9]:
# Feature of food101 dataset from TFDS
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [11]:
# Get the classnames
class_names = ds_info.features["label"].names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [13]:
# Take one sample of the train data
train_one_sample = train_data.take(1)
train_one_sample

<TakeDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [14]:
 # Output info about our training sample
 for image, label in train_one_sample:
   print(f"""
   image shape: {image.shape}
   image datatype: {image.dtype}
   Target class for fod101: {label}
   Class name: {class_names[label.numpy()]}
   """)


  image shape: (512, 512, 3)
  image datatype: <dtype: 'uint8'>
  Target class for fod101: 23
  Class name: churros
  


In [15]:
# What does our image tensor from tfds looks like
image 

<tf.Tensor: shape=(512, 512, 3), dtype=uint8, numpy=
array([[[ 93,  34,  40],
        [ 93,  34,  40],
        [ 94,  35,  41],
        ...,
        [ 52,  15,  23],
        [ 52,  15,  23],
        [ 52,  15,  23]],

       [[ 93,  34,  40],
        [ 93,  34,  40],
        [ 94,  35,  41],
        ...,
        [ 52,  15,  23],
        [ 52,  15,  23],
        [ 52,  15,  23]],

       [[ 93,  34,  40],
        [ 93,  34,  40],
        [ 94,  35,  41],
        ...,
        [ 54,  14,  23],
        [ 52,  15,  23],
        [ 52,  15,  23]],

       ...,

       [[108,  31,  49],
        [106,  29,  47],
        [105,  28,  46],
        ...,
        [121,  38,  56],
        [122,  39,  59],
        [123,  40,  60]],

       [[107,  30,  48],
        [107,  30,  48],
        [107,  30,  48],
        ...,
        [121,  38,  58],
        [123,  40,  60],
        [123,  40,  60]],

       [[106,  29,  47],
        [108,  31,  49],
        [109,  32,  50],
        ...,
        [123,  40,  6

In [16]:
# What is the maximum and minimum values
import tensorflow as tf
tf.reduce_max(image), tf.reduce_min(image)

(<tf.Tensor: shape=(), dtype=uint8, numpy=255>,
 <tf.Tensor: shape=(), dtype=uint8, numpy=9>)